Ya begitu la ya, 

<h1>Mobility Data</h1>
from the following :
<li> https://www.google.com/covid19/mobility
<li> https://covid19.apple.com/mobility
<li> https://dataforgood.fb.com/docs/covid19
<h2>2020 ID Region Mobility Report by Google</h2>

In [55]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [56]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.impute import SimpleImputer
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

In [57]:
Mobility_ID = pd.read_csv('/content/drive/MyDrive/Data Science Study/Projects/2020_ID_Region_Mobility_Report.csv')
Mobility_ID.sample(5)

,country_region_code,country_region,sub_region_1,sub_region_2,metro_area,iso_3166_2_code,census_fips_code,place_id,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
10471,ID,Indonesia,West Papua,NaN,NaN,ID-PB,NaN,ChIJLQviub0KVC0RYsvHxfjBSVM,2020-09-01,-5.0,NaN,NaN,NaN,-15,7
8029,ID,Indonesia,South Kalimantan,NaN,NaN,ID-KS,NaN,ChIJRbTSvTm33S0RE8GXt1C2fhQ,2020-02-19,-2.0,-2.0,-4.0,-5.0,4,1
4796,ID,Indonesia,Jambi,NaN,NaN,ID-JA,NaN,ChIJ2aytzd2IJS4R8E-Q0103c3Q,2020-12-13,-12.0,10.0,9.0,-19.0,-3,7
5048,ID,Indonesia,Lampung,NaN,NaN,ID-LA,NaN,ChIJpyKsUwF2Oy4RmrCJX8dYO48,2020-10-05,-8.0,15.0,-9.0,-25.0,-30,5
364,ID,Indonesia,Aceh,NaN,NaN,ID-AC,NaN,ChIJvcR8UN-bOTARYMogsoCdAwE,2020-03-29,-39.0,-21.0,-47.0,-43.0,-10,13


In [58]:
Mobility_ID.shape

(11235, 15)

In [59]:
Mobility_ID.isnull().sum()

country_region_code                                       0
country_region                                            0
sub_region_1                                            321
sub_region_2                                          11235
metro_area                                            11235
iso_3166_2_code                                         321
census_fips_code                                      11235
place_id                                                  0
date                                                      0
retail_and_recreation_percent_change_from_baseline       50
grocery_and_pharmacy_percent_change_from_baseline        99
parks_percent_change_from_baseline                      118
transit_stations_percent_change_from_baseline           391
workplaces_percent_change_from_baseline                   0
residential_percent_change_from_baseline                  0
dtype: int64

In [60]:
# removing null coloumns
Mobility_ID.drop(labels=['sub_region_2','metro_area','census_fips_code'],axis=1, inplace=True)
Mobility_ID.isnull().sum()

country_region_code                                     0
country_region                                          0
sub_region_1                                          321
iso_3166_2_code                                       321
place_id                                                0
date                                                    0
retail_and_recreation_percent_change_from_baseline     50
grocery_and_pharmacy_percent_change_from_baseline      99
parks_percent_change_from_baseline                    118
transit_stations_percent_change_from_baseline         391
workplaces_percent_change_from_baseline                 0
residential_percent_change_from_baseline                0
dtype: int64

In [61]:
Mobility_ID.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11235 entries, 0 to 11234
Data columns (total 12 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   country_region_code                                 11235 non-null  object 
 1   country_region                                      11235 non-null  object 
 2   sub_region_1                                        10914 non-null  object 
 3   iso_3166_2_code                                     10914 non-null  object 
 4   place_id                                            11235 non-null  object 
 5   date                                                11235 non-null  object 
 6   retail_and_recreation_percent_change_from_baseline  11185 non-null  float64
 7   grocery_and_pharmacy_percent_change_from_baseline   11136 non-null  float64
 8   parks_percent_change_from_baseline                  11117 non-null  float64


In [62]:
Mobility_ID.describe()

,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
count,11185.000000,11136.000000,11117.000000,10844.000000,11235.000000,11235.00000
mean,-17.864193,-3.340876,-13.457857,-33.490686,-18.755496,8.48073
std,14.183591,13.504384,19.937717,20.081918,14.505030,5.69989
min,-89.000000,-92.000000,-96.000000,-95.000000,-89.000000,-9.00000
25%,-27.000000,-12.000000,-24.000000,-48.000000,-26.000000,5.00000
50%,-16.000000,-2.000000,-11.000000,-34.000000,-20.000000,9.00000
75%,-8.000000,5.000000,-1.000000,-21.000000,-11.000000,12.00000
max,25.000000,74.000000,99.000000,55.000000,24.000000,37.00000


<h2>Mobility by Facebook</h2>

In [63]:
movement_range = pd.read_csv('/content/drive/MyDrive/Data Science Study/Projects/movement-range-2021-08-21.txt', delimiter='\t')
movement_range.sample(5)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ds,country,polygon_source,polygon_id,polygon_name,all_day_bing_tiles_visited_relative_change,all_day_ratio_single_tile_users,baseline_name,baseline_type
2700229,2021-01-14,THA,GADM,THA.68.13_1,Samrong Thap,-0.22253,0.21770,full_february,DAY_OF_WEEK
2115286,2021-04-24,MYS,GADM,MYS.14.11_1,Kuching,-0.12113,0.16469,full_february,DAY_OF_WEEK
1573919,2021-02-23,JAM,GADM,JAM.11_1,Saint Mary,-0.12281,0.28691,full_february,DAY_OF_WEEK
2857234,2021-08-18,USA,FIPS,6113,Yolo,-0.05212,0.15126,full_february_except_presidents_day,DAY_OF_WEEK
1388972,2021-03-30,IMN,GADM,IMN.5.2_1,Douglas,-0.25220,0.29405,full_february,DAY_OF_WEEK


In [64]:
movement_range.isnull().sum() , movement_range.shape

(ds                                                 0
 country                                            0
 polygon_source                                     0
 polygon_id                                         0
 polygon_name                                  152977
 all_day_bing_tiles_visited_relative_change         0
 all_day_ratio_single_tile_users                    0
 baseline_name                                      0
 baseline_type                                      0
 dtype: int64, (3506275, 9))

In [65]:
# get Indonesian data only
movement_range_ID = movement_range[movement_range['country'] == 'IDN'] 

In [66]:
movement_range_ID.head()

,ds,country,polygon_source,polygon_id,polygon_name,all_day_bing_tiles_visited_relative_change,all_day_ratio_single_tile_users,baseline_name,baseline_type
1277437,2021-01-01,IDN,GADM,IDN.1.10_1,Aceh Timur,-0.00045,0.25399,full_february,DAY_OF_WEEK
1277438,2021-01-02,IDN,GADM,IDN.1.10_1,Aceh Timur,-0.05155,0.24741,full_february,DAY_OF_WEEK
1277439,2021-01-03,IDN,GADM,IDN.1.10_1,Aceh Timur,-0.10062,0.28182,full_february,DAY_OF_WEEK
1277440,2021-01-04,IDN,GADM,IDN.1.10_1,Aceh Timur,0.02546,0.20491,full_february,DAY_OF_WEEK
1277441,2021-01-05,IDN,GADM,IDN.1.10_1,Aceh Timur,-0.01233,0.21060,full_february,DAY_OF_WEEK


In [67]:
movement_range_ID.shape, movement_range_ID.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 111447 entries, 1277437 to 1388883
Data columns (total 9 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   ds                                          111447 non-null  object 
 1   country                                     111447 non-null  object 
 2   polygon_source                              111447 non-null  object 
 3   polygon_id                                  111447 non-null  object 
 4   polygon_name                                111447 non-null  object 
 5   all_day_bing_tiles_visited_relative_change  111447 non-null  float64
 6   all_day_ratio_single_tile_users             111447 non-null  float64
 7   baseline_name                               111447 non-null  object 
 8   baseline_type                               111447 non-null  object 
dtypes: float64(2), object(7)
memory usage: 8.5+ MB


((111447, 9), None)

In [68]:
# drop uneccesary data
movement_range_ID.drop(labels=['country','polygon_source','polygon_id','baseline_name','baseline_type'], axis = 1 , inplace=True)
movement_range_ID.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 111447 entries, 1277437 to 1388883
Data columns (total 4 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   ds                                          111447 non-null  object 
 1   polygon_name                                111447 non-null  object 
 2   all_day_bing_tiles_visited_relative_change  111447 non-null  float64
 3   all_day_ratio_single_tile_users             111447 non-null  float64
dtypes: float64(2), object(2)
memory usage: 4.3+ MB


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [69]:
movement_range_ID.rename(columns={'ds':'tanggal',
                                   'polygon_name':'Kabupaten'}, inplace=True)


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [71]:
# save to excel 
movement_range_ID.to_excel("/content/drive/MyDrive/Data Science Study/Projects/movement_range_ID_preprocessed.xlsx", 
                           index = True, header = True)